In [2]:
!del hawaii.sqlite

Could Not Find C:\Users\ravi_\OneDrive\Documents\Rutgers Data Science Course\Homework\Week10 - ASQL\hawaii.sqlite


In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [4]:
# Create an engine to a SQLite database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [5]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [6]:
# Dependencies
import pandas as pd
import numpy as np
import os

In [7]:
# Path of the CSV files to read data from
hi_stations_csvfile = "Resources\hawaii_stations.csv"
hi_msrmnts_csvfile = "Resources\clean_hawaii_measurements.csv"

In [8]:
# Read CSV files into  pandas DataFrames
stations_df = pd.read_csv(hi_stations_csvfile, dtype=object)
msrmnts_df = pd.read_csv(hi_msrmnts_csvfile, dtype=object)

In [9]:
# Preview DataFrame - first stations

stations_df

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.3152,-157.9992,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.3331,-157.8025,152.4


In [10]:
# Preview DataFrame - Now measurements
msrmnts_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0,63
2,USC00519397,2010-01-03,0,74
3,USC00519397,2010-01-04,0,76
4,USC00519397,2010-01-07,0.06,70


In [11]:
##Create an ORM class for stations
Base = declarative_base()

class Station(Base):
    __tablename__ = 'stations'
    station = Column(Text,primary_key=True)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
   
    def __repr__(self):
        return f"id={self.id}, name={self.name}"

In [12]:
##Create an ORM class for measurements


class Measurement(Base):
    __tablename__ = 'measurements'
    station = Column(Text,primary_key=True)
    date = Column(Text,primary_key=True)
    prcp = Column(Float)
    tobs = Column(Integer)

In [13]:
# Use `create_all` to create the tables in the database
Base.metadata.create_all(engine)


In [14]:
##check the tables were created in sqlite
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

['measurements', 'stations']

In [15]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()


In [16]:
# Save the reference to the `stations` table as a variable called `stations_table`
station_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [17]:
# create a dictionary to write station data to sql table
stations_data = stations_df.to_dict(orient='records')

In [18]:
# Use `stations_table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(station_table.insert(), stations_data)

In [19]:
#confirm data insertion
# Test that the insert works by fetching the first all(8) rows. 
conn.execute("select * from stations ").fetchall()

[('USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 ('USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 ('USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 ('USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 ('USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6),
 ('USC00519523', 'WAIMANALO EXPERIMENTAL FARM, HI US', 21.33556, -157.71139, 19.5),
 ('USC00519281', 'WAIHEE 837.5, HI US', 21.45167, -157.84889, 32.9),
 ('USC00511918', 'HONOLULU OBSERVATORY 702.2, HI US', 21.3152, -157.9992, 0.9),
 ('USC00516128', 'MANOA LYON ARBO 785.2, HI US', 21.3331, -157.8025, 152.4)]

In [20]:
# Now save the measurments data to another table
msrmnts_table = sqlalchemy.Table('measurements', metadata, autoload=True)

In [21]:
# create a dictionary to write measurements data to sql table
msrmnts_data = msrmnts_df.to_dict(orient='records')

In [22]:
#Insert the data for measurements
conn.execute(msrmnts_table.insert(), msrmnts_data)

In [23]:
#confirm data insertion
# Test that the insert works by fetching the first first 5 rows. 
conn.execute("select * from measurements limit 5 ").fetchall()

[('USC00519397', '2010-01-01', 0.08, 65),
 ('USC00519397', '2010-01-02', 0.0, 63),
 ('USC00519397', '2010-01-03', 0.0, 74),
 ('USC00519397', '2010-01-04', 0.0, 76),
 ('USC00519397', '2010-01-07', 0.06, 70)]